In [1]:
import os, csv, json
from dotenv import load_dotenv
import time
from datetime import datetime
import schedule
import requests

import yfinance as yf
yf.pdr_override()

import pandas as pd
import pandas_ta as ta
import numpy as np

from google.oauth2 import service_account
import pygsheets

load_dotenv()

True

In [2]:
with open("..\service_account.json") as source:
    info = json.load(source)
credentials = service_account.Credentials.from_service_account_info(info)

CLIENT = pygsheets.authorize(service_account_file='..\service_account.json')
SHEET_URL = os.getenv('TRADE_SHEET')
data = CLIENT.sheet.get(os.getenv('TRADE_SHEET_ID'))
data['spreadsheetId']

'1WTJj8qkV9GeM_Fz8m2bLH3gZNNpSfRnqtzBiv3XaFsc'

In [3]:
LINE_TOKEN = os.getenv('LINE_TOKEN')
LINE_URL = 'https://notify-api.line.me/api/notify'
LINE_HEADERS = {'content-type':'application/x-www-form-urlencoded','Authorization':'Bearer ' + LINE_TOKEN }

In [4]:
def get_initial_bars(pair, tf):
    ticker_yahoo = yf.Ticker(pair+'=x')
    bars = ticker_yahoo.history(period='3d', interval=tf)
    df = pd.DataFrame({
        'datetime': bars.index,
        'open': bars['Open'],
        'high': bars['High'],
        'low': bars['Low'],
        'close': bars['Close'],
        'volumn': bars['Volume']
    }) # convert alpha vantage data to use with strategy

    df['timestamp'] = pd.to_datetime(df['datetime']).values.astype(np.int64) // 10**9
    df['datetime'] = pd.to_datetime(df['timestamp'] * 10**9, utc=True).dt.tz_convert('Asia/Bangkok')
    df.reset_index(drop=True, inplace=True)

    return df

In [17]:
d = get_initial_bars('EURUSD', '15m')
d

,datetime,open,high,low,close,volumn,timestamp
0,2022-07-22 06:00:00+07:00,1.022704,1.023018,1.022704,1.022704,0,1658444400
1,2022-07-22 06:15:00+07:00,1.022809,1.023018,1.022809,1.022913,0,1658445300
2,2022-07-22 06:30:00+07:00,1.022913,1.023437,1.022913,1.023227,0,1658446200
3,2022-07-22 06:45:00+07:00,1.023123,1.023123,1.022390,1.022390,0,1658447100
4,2022-07-22 07:00:00+07:00,1.022495,1.022495,1.021555,1.021555,0,1658448000
...,...,...,...,...,...,...,...
201,2022-07-26 09:45:00+07:00,1.024380,1.024800,1.024275,1.024275,0,1658803500
202,2022-07-26 10:00:00+07:00,1.024170,1.024485,1.024170,1.024275,0,1658804400
203,2022-07-26 10:15:00+07:00,1.024066,1.024275,1.023856,1.023961,0,1658805300
204,2022-07-26 10:30:00+07:00,1.023961,1.024066,1.023541,1.023541,0,1658806200


In [25]:
# dt['ts'] = dt['timestamp'].dt.timestamp()
pd.to_datetime(dt['timestamp'], utc=True).dt.tz_localize(None)

0    2022-07-24 23:00:00
1    2022-07-24 23:15:00
2    2022-07-24 23:30:00
3    2022-07-24 23:45:00
4    2022-07-25 00:00:00
5    2022-07-25 00:15:00
6    2022-07-25 00:30:00
7    2022-07-25 00:45:00
8    2022-07-25 01:00:00
9    2022-07-25 01:15:00
10   2022-07-25 01:30:00
11   2022-07-25 01:45:00
12   2022-07-25 02:00:00
13   2022-07-25 02:15:00
14   2022-07-25 02:30:00
15   2022-07-25 02:45:00
16   2022-07-25 03:00:00
17   2022-07-25 03:15:00
18   2022-07-25 03:17:27
Name: timestamp, dtype: datetime64[ns]

In [73]:
def get_latest_bars(df, pair, tf):
    ticker_yahoo = yf.Ticker(pair+'=x')
    bars = ticker_yahoo.history(period='1d', interval=tf)
    incoming_df = pd.DataFrame({
        'datetime': bars.index,
        'open': bars['Open'],
        'high': bars['High'],
        'low': bars['Low'],
        'close': bars['Close'],
        'volumn': bars['Volume']
    }) # convert alpha vantage data to use with rsi strategy

    incoming_df['timestamp'] = pd.to_datetime(incoming_df['datetime']).values.astype(np.int64) // 10**9
    incoming_df['datetime'] = pd.to_datetime(incoming_df['timestamp'] * 10**9, utc=True).dt.tz_convert('Asia/Bangkok')
         
    if (incoming_df['timestamp'].iloc[0] != df['timestamp'].iloc[-1]):
        df = df.append(incoming_df.tail(1), ignore_index = True)
    return df

In [74]:
nd = get_latest_bars(d, 'EURUSD', '15m')
nd

,datetime,open,high,low,close,volumn,timestamp
0,2022-07-25 06:00:00+07:00,1.021555,1.021555,1.021033,1.021033,0,1658703600
1,2022-07-25 06:15:00+07:00,1.020825,1.020825,1.020512,1.020616,0,1658704500
2,2022-07-25 06:30:00+07:00,1.020616,1.020721,1.020408,1.020408,0,1658705400
3,2022-07-25 06:45:00+07:00,1.020408,1.020721,1.020200,1.020200,0,1658706300
4,2022-07-25 07:00:00+07:00,1.020408,1.020408,1.019472,1.019784,0,1658707200
5,2022-07-25 07:15:00+07:00,1.019888,1.019992,1.019784,1.019784,0,1658708100
6,2022-07-25 07:30:00+07:00,1.019576,1.019680,1.019160,1.019368,0,1658709000
7,2022-07-25 07:45:00+07:00,1.019472,1.019472,1.018537,1.018537,0,1658709900
8,2022-07-25 08:00:00+07:00,1.018641,1.019368,1.018641,1.019264,0,1658710800
9,2022-07-25 08:15:00+07:00,1.018952,1.019576,1.018849,1.018952,0,1658711700


In [4]:
def writeTradeLog(row, pair): # only xStation log only
    new_row = [] # convert data from Xstation.
    sheet = CLIENT.open_by_url(SHEET_URL)
    worksheet = sheet.worksheet_by_title('FowardTest')
    cells = worksheet.get_all_values(include_tailing_empty_rows=False, include_tailing_empty=False, returnas='matrix')
    last_row = len(cells)

    new_row.append([
        str(last_row), pair, row['action'], row['open time'], row['open'], row['close time'], row['close'],
        row['result'], row['strategy'], row['size']
    ])
    
    worksheet = worksheet.insert_rows(last_row, number=1, values= new_row)
    print(last_row, new_row)
    print('done...')

In [5]:
def notifyMsg(*msg):
    r = requests.post(LINE_URL, headers=LINE_HEADERS, data={'message': msg[0]})
    print(r.text)

In [6]:
class Psar_Bot:
    def __init__(self, params):
        self.client = None
        self.pair = params['pair']
        self.tf = params['tf']
        self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                      'close': None, 'T/P': None, 'S/L': None,'result': None, 'strategy': 'PsarReveral', 'size': 0}
        
        self.current_action = 'close'
        self.atr = params['atr']
        self.ema = params['ema']
        self.psar = params['psar']
        
        self.pip = params['pip']
        self.pip_value = params['pip_value']
        self.rr = params['rr']
        
        self.lot_size = 0.01
        self.order_id = 0
        self.target = 10
        self.round = params['round']
        
        self.df = None
        
    def reset_order(self):
        self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                      'close': None, 'T/P': None, 'S/L': None,'result': None, 'strategy': 'PsarReveral', 'size': 0}
    def get_bars(self, period_bars='3d'):
        ticker_yahoo = yf.Ticker(self.pair+'=x')
        bars = ticker_yahoo.history(period=period_bars, interval=self.tf)
        df = pd.DataFrame({
            'datetime': bars.index,
            'open': bars['Open'],
            'high': bars['High'],
            'low': bars['Low'],
            'close': bars['Close'],
            'volumn': bars['Volume']
        }) # convert alpha vantage data to use with strategy

        df['timestamp'] = pd.to_datetime(df['datetime']).values.astype(np.int64) // 10**9
        df['datetime'] = pd.to_datetime(df['timestamp'] * 10**9, utc=True).dt.tz_convert('Asia/Bangkok')
        df.reset_index(drop=True, inplace=True)
        return df
        
    def initial_bars(self):
        init_df = self.get_bars(period_bars='3d')
        self.df = init_df
        
    def calLotSize(self, entry, exit):
        self.lot_size = abs(round(self.target / (((entry - exit) / self.pip ) * self.pip_value ) , 2))
        print(f'lot_size: {str(self.lot_size)}')
        
    def longCondition(self, row, pre_row):
        if ~np.isnan(row['PSARl_'+str(self.psar)+'_'+str(self.psar*10)]) and \
            ~np.isnan(pre_row['PSARs_'+str(self.psar)+'_'+str(self.psar*10)]) and \
            (row['close'] > row['EMA_'+str(self.ema)]):
                    
            self.order['action'] = 'long'
            self.order['open time'] = row['timestamp']
            self.order['open'] = row['open']
                
            self.order['S/L'] = row['low']- row['ATRr_'+str(self.atr)]
            self.order['T/P'] = row['open'] + (self.rr * abs(row['open'] - self.order['S/L']))
            self.current_action = 'long'
            notifyMsg(f"Buy {self.pair} Psar\nprice: {str(self.order['open'])}\nTP: {str(self.order['T/P'])}\nSL: {str(self.order['S/L'])}")
                
            self.calLotSize(self.order['open'], self.order['T/P'])
                
    def longResult(self, row):
        if (row['low'] <= self.order['S/L']):
            self.order['result'] = 'S/L'
            self.order['close time'] = row['timestamp']
            self.order['close'] = self.order['S/L']
                
            self.current_action = 'close'
            notifyMsg(f"S/L Buy {self.pair} Psar\nprice: {self.order['close']}")
            self.order['size'] = self.lot_size
            writeTradeLog(self.order, self.pair)
                
            self.reset_order()
                
        elif (row['high'] >= self.order['T/P']):
            self.order['result'] = 'T/P'
            self.order['close time'] = row['timestamp']
            self.order['close'] = self.order['T/P']
                
            self.current_action = 'close'
            notifyMsg(f"T/P Buy {self.pair} Psar\nprice: {self.order['close']}")
            self.order['size'] = self.lot_size
            writeTradeLog(self.order, self.pair)
                
            self.reset_order()
        
    def shortCondition(self, row, pre_row):
        if ~np.isnan(row['PSARs_'+str(self.psar)+'_'+str(self.psar*10)]) and \
            ~np.isnan(pre_row['PSARl_'+str(self.psar)+'_'+str(self.psar*10)]) and \
            (row['close'] < row['EMA_'+str(self.ema)]):

            self.order['action'] = 'short'
            self.order['open time'] = row['timestamp']
            self.order['open'] = row['open']
                
            self.order['S/L'] = row['high'] + row['ATRr_'+str(self.atr)]
            self.order['T/P'] = row['open'] - (self.rr * abs(row['open'] - self.order['S/L']))
            self.current_action = 'short'
            notifyMsg(f"Sell {self.pair} Psar\nprice: {str(self.order['open'])}\nTP: {str(self.order['T/P'])}\nSL: {str(self.order['S/L'])}")
                
            self.calLotSize(self.order['open'], self.order['T/P'])
                
    def shortResult(self, row):
        if (row['high'] >= self.order['S/L']):
            self.order['result'] = 'S/L'
            self.order['close time'] = row['timestamp']
            self.order['close'] = self.order['S/L']
                
            self.current_action = 'close'
            notifyMsg(f"S/L Sell {self.pair} Psar\nprice: {self.order['close']}")
            self.order['size'] = self.lot_size
            writeTradeLog(self.order, self.pair)
                
            self.reset_order()
                
        elif (row['low'] <= self.order['T/P']):
            self.order['result'] = 'T/P'
            self.order['close time'] = row['timestamp']
            self.order['close'] = self.order['T/P']
                
            self.current_action = 'close'
            notifyMsg(f"T/P Sell {self.pair} Psar\nprice: {self.order['close']}")
            self.order['size'] = self.lot_size
            writeTradeLog(self.order, self.pair)
                
            self.reset_order()
        
    def createStrategy(self):
        parabolic_sar = ta.Strategy(
            name = 'Parabolic Sar',
            description = 'Parabolic Sar Reversal',
            ta = [
                {'kind': 'psar', 'af0': self.psar, 'af': self.psar, 'max_af': self.psar*10},
                {'kind': 'ema', 'length': self.ema},
                {'kind': 'atr', 'length': self.atr}
            ]
        )
        self.df.ta.strategy(parabolic_sar)
        
    def realtime_long(self):
        incoming_df = self.get_bars(period_bars='1d')
        if (incoming_df['timestamp'].iloc[0] != self.df['timestamp'].iloc[-1]):
            self.df = self.df.append(incoming_df.tail(1), ignore_index = True)
        self.createStrategy()
        
        row = self.df.tail(1).to_dict('records')[0]
        pre_row = self.df.tail(2).to_dict('records')[0]
        if self.current_action == 'close':
            self.longCondition(row, pre_row)
        else:
            self.longResult(row)
    
    def realtime_short(self):
        incoming_df = self.get_bars(period_bars='1d')
        if (incoming_df['timestamp'].iloc[0] != self.df['timestamp'].iloc[-1]):
            self.df = self.df.append(incoming_df.tail(1), ignore_index = True)
        self.createStrategy()
        
        row = self.df.tail(1).to_dict('records')[0]
        pre_row = self.df.tail(2).to_dict('records')[0]
        if self.current_action == 'close':
            self.shortCondition(row, pre_row)
        else:
            self.shortResult(row)
    
    def realtime_both(self):
        incoming_df = self.get_bars(period_bars='1d')
        if (incoming_df['timestamp'].iloc[0] != self.df['timestamp'].iloc[-1]):
            self.df = self.df.append(incoming_df.tail(1), ignore_index = True)
        self.createStrategy()
        
        row = self.df.tail(1).to_dict('records')[0]
        pre_row = self.df.tail(2).to_dict('records')[0]
        if self.current_action == 'close':
            self.longCondition(row, pre_row)
            self.shortCondition(row, pre_row)
            
        elif self.current_action == 'long':
            self.longResult(row)
                    
        elif self.current_action == 'short':
            self.shortResult(row)
        
#     def get_realtime(self):
#         ticker_yahoo = yf.Ticker(self.pair+'=x')
#         bars = ticker_yahoo.history(period='1d', interval=self.tf)
#         incoming_df = pd.DataFrame({
#             'datetime': bars.index,
#             'open': bars['Open'],
#             'high': bars['High'],
#             'low': bars['Low'],
#             'close': bars['Close'],
#             'volumn': bars['Volume']
#         }) # convert alpha vantage data to use with rsi strategy

#         incoming_df['timestamp'] = pd.to_datetime(incoming_df['datetime']).values.astype(np.int64) // 10**9
#         incoming_df['datetime'] = pd.to_datetime(incoming_df['timestamp'] * 10**9, utc=True).dt.tz_convert('Asia/Bangkok')

#         if (incoming_df['timestamp'].iloc[0] != self.df['timestamp'].iloc[-1]):
#             self.df = self.df.append(incoming_df.tail(1), ignore_index = True)
        
#         parabolic_sar = ta.Strategy(
#             name = 'Parabolic Sar',
#             description = 'Parabolic Sar Reversal',
#             ta = [
#                 {'kind': 'psar', 'af0': self.psar, 'af': self.psar, 'max_af': self.psar*10},
#                 {'kind': 'ema', 'length': self.ema},
#                 {'kind': 'atr', 'length': self.atr}
#             ]
#         )
#         self.df.ta.strategy(parabolic_sar)
    
#         row = self.df.tail(1).to_dict('records')[0]
#         pre_row = self.df.tail(2).to_dict('records')[0]
        
#         if self.current_action == 'close':
                
#             if ~np.isnan(row['PSARl_'+str(self.psar)+'_'+str(self.psar*10)]) and \
#                 ~np.isnan(pre_row['PSARs_'+str(self.psar)+'_'+str(self.psar*10)]) and \
#                 (row['close'] > row['EMA_'+str(self.ema)]):
                    
#                 self.order['action'] = 'buy'
#                 self.order['open time'] = row['timestamp']
#                 self.order['open'] = row['open']
                
#                 self.order['S/L'] = row['low']- row['ATRr_'+str(self.atr)]
#                 self.order['T/P'] = row['open'] + (self.rr * abs(row['open'] - self.order['S/L']))
#                 self.current_action = 'buy'
#                 notifyMsg(f"Buy {self.pair} Psar\nprice: {str(self.order['open'])}\nTP: {str(self.order['T/P'])}\nSL: {str(self.order['S/L'])}")
                
#                 self.calLotSize(self.order['open'], self.order['T/P'])
                
#             elif ~np.isnan(row['PSARs_'+str(self.psar)+'_'+str(self.psar*10)]) and \
#                 ~np.isnan(pre_row['PSARl_'+str(self.psar)+'_'+str(self.psar*10)]) and \
#                 (row['close'] < row['EMA_'+str(self.ema)]):

#                 self.order['action'] = 'sell'
#                 self.order['open time'] = row['timestamp']
#                 self.order['open'] = row['open']
                
#                 self.order['S/L'] = row['high'] + row['ATRr_'+str(self.atr)]
#                 self.order['T/P'] = row['open'] - (self.rr * abs(row['open'] - self.order['S/L']))
#                 self.current_action = 'sell'
#                 notifyMsg(f"Sell {self.pair} Psar\nprice: {str(self.order['open'])}\nTP: {str(self.order['T/P'])}\nSL: {str(self.order['S/L'])}")
                
#                 self.calLotSize(self.order['open'], self.order['T/P'])

#         if (self.current_action == 'buy'):
#             if (row['low'] <= self.order['S/L']):
#                 self.order['result'] = 'S/L'
#                 self.order['close time'] = row['timestamp']
#                 self.order['close'] = self.order['S/L']
                
#                 self.current_action = 'close'
#                 notifyMsg(f"S/L Buy {self.pair} Psar\nprice: {self.order['close']}")
#                 self.order['size'] = self.lot_size
#                 writeTradeLog(self.order, self.pair)
                
#                 self.reset_order()
                
#             elif (row['high'] >= self.order['T/P']):
#                 self.order['result'] = 'T/P'
#                 self.order['close time'] = row['timestamp']
#                 self.order['close'] = self.order['T/P']
                
#                 self.current_action = 'close'
#                 notifyMsg(f"T/P Buy {self.pair} Psar\nprice: {self.order['close']}")
#                 self.order['size'] = self.lot_size
#                 writeTradeLog(self.order, self.pair)
                
#                 self.reset_order()

#         elif (self.current_action == 'sell'):
#             if (row['high'] >= self.order['S/L']):
#                 self.order['result'] = 'S/L'
#                 self.order['close time'] = row['timestamp']
#                 self.order['close'] = self.order['S/L']
                
#                 self.current_action = 'close'
#                 notifyMsg(f"S/L Sell {self.pair} Psar\nprice: {self.order['close']}")
#                 self.order['size'] = self.lot_size
#                 writeTradeLog(self.order, self.pair)
                
#                 self.reset_order()
                
#             elif (row['low'] <= self.order['T/P']):
#                 self.order['result'] = 'T/P'
#                 self.order['close time'] = row['timestamp']
#                 self.order['close'] = self.order['T/P']
                
#                 self.current_action = 'close'
#                 notifyMsg(f"T/P Sell {self.pair} Psar\nprice: {self.order['close']}")
#                 self.order['size'] = self.lot_size
#                 writeTradeLog(self.order, self.pair)
                
#                 self.reset_order()

In [7]:
eurusd_psar = Psar_Bot({
    'pair': 'EURUSD',
    'atr': 17,
    'psar': 0.01,
    'ema': 200,
    'pip': 0.0001,
    'pip_value': 10,
    'rr': 1.5,
    'round': 5,
    'tf': '15m',
})

In [8]:
eurusd_psar.initial_bars()

In [9]:
def get_all_realtime():
#     eurusd_psar.get_realtime()
    eurusd_psar.realtime_both()
    
schedule.every(60 * 15).seconds.do(get_all_realtime) 

Every 900 seconds do get_all_realtime() (last run: [never], next run: 2022-08-26 09:32:52)

In [ ]:
while True:
    schedule.run_pending()
    time.sleep(1)

{"status":200,"message":"ok"}
lot_size: 0.15
{"status":200,"message":"ok"}
20 [['20', 'EURUSD', 'long', 1661481113, 0.9975062012672424, 1661489271, 0.9970566139618766, 'S/L', 'PsarReveral', 0.15]]
done...
{"status":200,"message":"ok"}
lot_size: 0.21
{"status":200,"message":"ok"}
21 [['21', 'EURUSD', 'short', 1661497374, 0.9957184195518494, 1661499175, 0.9960305395160446, 'S/L', 'PsarReveral', 0.21]]
done...
{"status":200,"message":"ok"}
lot_size: 0.19
{"status":200,"message":"ok"}
22 [['22', 'EURUSD', 'long', 1661501931, 0.9979044198989868, 1661503732, 0.9984191660292587, 'T/P', 'PsarReveral', 0.19]]
done...


In [11]:
wt = {'action': 'sell', 'open time': 1658902646, 'open': 1.0140959024429321, 'close time': 1658913445, 'close': 1.0146209342649926, 'T/P': 1.0133083547098414, 'S/L': 1.0146209342649926, 'result': 'S/L', 'strategy': 'PsarReveral', 'size': 0.13}
wt

{'action': 'sell',
 'open time': 1658902646,
 'open': 1.0140959024429321,
 'close time': 1658913445,
 'close': 1.0146209342649926,
 'T/P': 1.0133083547098414,
 'S/L': 1.0146209342649926,
 'result': 'S/L',
 'strategy': 'PsarReveral',
 'size': 0.13}

In [40]:
writeTradeLog(wt, 'EURUSD')

1 [['1', 'EURUSD', 'sell', 1658902646, 1.0140959024429321, 1658913445, 1.0146209342649926, 'S/L', 'PsarReveral', 0.13]]
done...
